In [ ]:
!pip install selenium
!pip install rasterio
!pip install rasterstats
!pip install geopandas
!pip install folium
!apt install firefox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 20.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.8/136.8 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 69.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from PIL import Image
import io

# set options for Firefox webdriver
options = Options()
options.headless = True

# initialize Firefox webdriver
driver = webdriver.Firefox(options=options)

# set path for geckodriver executable
driver_path = 'geckodriver'

# set executable path for Firefox webdriver
driver = webdriver.Firefox(executable_path=driver_path, options=options)


<ipython-input-7-a727aed52abe>:8: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = True



**Creating a Folium map centered on the median of the GPS data and adding a marker for each point in the GeoDataFrame**

In [ ]:
import geopandas as gpd
gps_df = pd.read_csv("/content/gps_locations.csv")
geo_df = gpd.GeoDataFrame(gps_df, geometry=gpd.points_from_xy(gps_df.longitude, gps_df.latitude))
center = [geo_df['latitude'].median(), geo_df['longitude'].median()]
map_ = folium.Map(location=center, zoom_start=10, tiles='OpenStreetMap')
for _, row in geo_df.iterrows():
 folium.Marker(location=[row['latitude'], row['longitude']]).add_to(map_)
image = map_._to_png(5)
img = Image.open(io.BytesIO(image))
img.save('9pointsmap.png') 
map_

**Adding a marker and circle for each point in the geopandas dataframe**

In [ ]:
center_coords = [geo_df['latitude'].median(), geo_df['longitude'].median()]
folium_map = folium.Map(location=center_coords, zoom_start=10, tiles='OpenStreetMap')
for _, row in geo_df.iterrows():
   folium.Marker(location=[row['latitude'], row['longitude']], icon=folium.Icon(color='blue')).add_to(folium_map)
   folium.Circle(location=[row['geometry'].y, row['geometry'].x], radius=100, color='red', fill=True, fill_color='red').add_to(folium_map)
image = folium_map._to_png(6)
img = Image.open(io.BytesIO(image))
img.save('9_pointsbuffer_map.png')
folium_map

**Adding imperviousness data to the GPS locations**

In [ ]:
import rasterio
from rasterstats import zonal_stats, point_query
geo_df['geometry'] = geo_df.buffer(100)
raster_file = '/content/NLCD_2016_Impervious_L48_20190405_aeCiajHeNAvXB2754sYm.tiff'
with rasterio.open(raster_file) as src:
 affine_transform = src.transform
 array = src.read(1)

stats = zonal_stats(geo_df, array, affine=affine_transform, stats='mean')
imperviousness_values = [stat['mean'] for stat in stats]
geo_df['imperviousness'] = imperviousness_values
geo_df.to_excel('gps_locations_with_imperviousness.xlsx', index=False)

print(geo_df)

   location_name   latitude  longitude  \
0              1  29.655960 -95.471525   
1              2  29.675354 -95.521851   
2              3  29.610736 -95.627920   
3              4  29.626681 -95.667396   
4              5  29.761629 -95.444837   
5              6  29.818425 -95.622570   
6              7  29.757133 -95.360462   
7              8  29.717137 -95.401621   
8              9  29.550501 -95.097477   

                                            geometry  imperviousness  
0  POLYGON ((4.528 29.656, 4.047 19.854, 2.607 10...            25.0  
1  POLYGON ((4.478 29.675, 3.997 19.874, 2.557 10...            25.0  
2  POLYGON ((4.372 29.611, 3.891 19.809, 2.451 10...            25.0  
3  POLYGON ((4.333 29.627, 3.851 19.825, 2.411 10...            25.0  
4  POLYGON ((4.555 29.762, 4.074 19.960, 2.634 10...            25.0  
5  POLYGON ((4.377 29.818, 3.896 20.017, 2.456 10...            25.0  
6  POLYGON ((4.640 29.757, 4.158 19.955, 2.718 10...            25.0  
7  POLYGON 